# Lab Exercise: Naive Bayes Text Classification & N-gram Language Models

**Course:** BUS 405 - Foundations of Big Data Analytics  
**Institution:** Pokhara University

---

## Learning Objectives

By the end of this lab, you will be able to:

1. Implement Naive Bayes classifier from scratch with Laplace smoothing
2. Understand and build N-gram language models (unigram, bigram, trigram)
3. Combine N-gram features with Naive Bayes for improved text classification
4. Evaluate classifier performance using various metrics
5. Apply these techniques to real-world text classification problems

---

## Part 1: Setup and Imports

In [ ]:
# Core libraries
import numpy as np
import pandas as pd
from collections import defaultdict, Counter
import re
import math

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Sklearn for comparison and metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report
)
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

# Set display options
pd.set_option('display.max_colwidth', 100)
plt.style.use('seaborn-v0_8-whitegrid')

print("All libraries imported successfully!")

---

## Part 2: Understanding Naive Bayes Classification

### 2.1 Theory Review

**Bayes' Theorem:**
$$P(Class|Document) = \frac{P(Document|Class) \times P(Class)}{P(Document)}$$

**Naive Assumption:** All features (words) are conditionally independent:
$$P(w_1, w_2, ..., w_n|Class) = \prod_{i=1}^{n} P(w_i|Class)$$

**Laplace Smoothing:**
$$P(w_i|Class) = \frac{count(w_i, Class) + k}{total\_words\_in\_Class + k \times |V|}$$

### 2.2 Sample Dataset: Nepali Business Review Classification

Let's create a sample dataset with business reviews categorized as **Positive** or **Negative**.

In [ ]:
# Sample dataset - Business reviews (English for simplicity)
training_data = [
    # Positive reviews
    ("excellent service great food highly recommend", "positive"),
    ("amazing experience wonderful staff friendly service", "positive"),
    ("best restaurant excellent quality delicious food", "positive"),
    ("great value good prices excellent location", "positive"),
    ("wonderful atmosphere friendly staff highly recommend", "positive"),
    ("outstanding service great food amazing experience", "positive"),
    ("excellent product fast delivery good quality", "positive"),
    ("highly recommend great service best prices", "positive"),
    
    # Negative reviews
    ("terrible service bad food never again", "negative"),
    ("worst experience rude staff poor quality", "negative"),
    ("bad service slow delivery disappointed", "negative"),
    ("poor quality overpriced terrible experience", "negative"),
    ("horrible food bad service waste money", "negative"),
    ("disappointing experience poor staff bad quality", "negative"),
    ("never recommend terrible service bad food", "negative"),
    ("awful experience worst service poor quality", "negative"),
]

# Test data
test_data = [
    ("great service excellent food", "positive"),
    ("bad experience terrible service", "negative"),
    ("good quality friendly staff", "positive"),
    ("poor service disappointed", "negative"),
]

# Display as DataFrame
df_train = pd.DataFrame(training_data, columns=['text', 'label'])
print("Training Data:")
print(df_train)
print(f"\nClass distribution: {df_train['label'].value_counts().to_dict()}")

---

## Part 3: Implementing Naive Bayes from Scratch

### 3.1 Text Preprocessing

In [ ]:
def preprocess_text(text):
    """
    Basic text preprocessing:
    - Convert to lowercase
    - Remove special characters
    - Tokenize into words
    """
    # Convert to lowercase
    text = text.lower()
    
    # Remove special characters and digits
    text = re.sub(r'[^a-z\s]', '', text)
    
    # Tokenize
    tokens = text.split()
    
    return tokens

# Test preprocessing
sample_text = "Excellent Service! Great food... Highly Recommend!!!"
print(f"Original: {sample_text}")
print(f"Preprocessed: {preprocess_text(sample_text)}")

### 3.2 Naive Bayes Classifier Class

In [ ]:
class NaiveBayesClassifier:
    """
    Multinomial Naive Bayes Classifier with Laplace Smoothing
    """
    
    def __init__(self, k=1):
        """
        Initialize the classifier.
        
        Parameters:
        -----------
        k : float
            Smoothing parameter (default=1 for Laplace smoothing)
        """
        self.k = k  # Smoothing parameter
        self.class_priors = {}  # P(class)
        self.word_counts = {}  # count(word, class)
        self.class_word_totals = {}  # total words per class
        self.vocabulary = set()  # unique words
        self.classes = []
        
    def fit(self, documents, labels):
        """
        Train the classifier on the given documents and labels.
        
        Parameters:
        -----------
        documents : list of str
            Training documents
        labels : list of str
            Corresponding labels
        """
        self.classes = list(set(labels))
        n_docs = len(documents)
        
        # Initialize data structures
        for c in self.classes:
            self.word_counts[c] = defaultdict(int)
            self.class_word_totals[c] = 0
        
        # Count documents per class and words per class
        class_doc_counts = Counter(labels)
        
        for doc, label in zip(documents, labels):
            tokens = preprocess_text(doc)
            for word in tokens:
                self.vocabulary.add(word)
                self.word_counts[label][word] += 1
                self.class_word_totals[label] += 1
        
        # Calculate class priors: P(class) = count(class) / total_docs
        for c in self.classes:
            self.class_priors[c] = class_doc_counts[c] / n_docs
            
        print(f"Training completed!")
        print(f"Vocabulary size: {len(self.vocabulary)}")
        print(f"Class priors: {self.class_priors}")
        
    def _calculate_word_probability(self, word, class_label):
        """
        Calculate P(word|class) with Laplace smoothing.
        
        Formula: P(w|c) = (count(w,c) + k) / (total_words_c + k * |V|)
        """
        word_count = self.word_counts[class_label].get(word, 0)
        total_words = self.class_word_totals[class_label]
        vocab_size = len(self.vocabulary)
        
        probability = (word_count + self.k) / (total_words + self.k * vocab_size)
        return probability
    
    def predict_proba(self, document, verbose=False):
        """
        Calculate probability of each class for a document.
        
        Returns dict of {class: probability}
        """
        tokens = preprocess_text(document)
        class_scores = {}
        
        for c in self.classes:
            # Start with log of prior probability
            log_prob = math.log(self.class_priors[c])
            
            if verbose:
                print(f"\nClass: {c}")
                print(f"  Prior P({c}) = {self.class_priors[c]:.4f}")
                print(f"  Log prior = {log_prob:.4f}")
            
            # Add log probabilities of each word
            for word in tokens:
                word_prob = self._calculate_word_probability(word, c)
                log_prob += math.log(word_prob)
                
                if verbose:
                    print(f"  P({word}|{c}) = {word_prob:.4f}, log = {math.log(word_prob):.4f}")
            
            class_scores[c] = log_prob
            
            if verbose:
                print(f"  Total log probability: {log_prob:.4f}")
        
        # Convert log probabilities to probabilities (normalized)
        max_log = max(class_scores.values())
        exp_scores = {c: math.exp(s - max_log) for c, s in class_scores.items()}
        total = sum(exp_scores.values())
        probabilities = {c: s / total for c, s in exp_scores.items()}
        
        return probabilities
    
    def predict(self, document):
        """
        Predict the class of a document.
        """
        probabilities = self.predict_proba(document)
        return max(probabilities, key=probabilities.get)
    
    def evaluate(self, documents, labels):
        """
        Evaluate the classifier on test data.
        """
        predictions = [self.predict(doc) for doc in documents]
        
        accuracy = accuracy_score(labels, predictions)
        precision = precision_score(labels, predictions, pos_label='positive')
        recall = recall_score(labels, predictions, pos_label='positive')
        f1 = f1_score(labels, predictions, pos_label='positive')
        
        return {
            'accuracy': accuracy,
            'precision': precision,
            'recall': recall,
            'f1_score': f1,
            'predictions': predictions
        }

print("NaiveBayesClassifier class defined successfully!")

### 3.3 Training and Testing the Classifier

In [ ]:
# Prepare training data
train_docs = [text for text, label in training_data]
train_labels = [label for text, label in training_data]

# Initialize and train the classifier
nb_classifier = NaiveBayesClassifier(k=1)  # k=1 for Laplace smoothing
nb_classifier.fit(train_docs, train_labels)

In [ ]:
# Detailed prediction example with step-by-step calculation
test_doc = "great service excellent food"
print(f"Testing document: '{test_doc}'")
print("=" * 60)

probabilities = nb_classifier.predict_proba(test_doc, verbose=True)

print("\n" + "=" * 60)
print("\nFinal Probabilities:")
for c, prob in probabilities.items():
    print(f"  P({c}|document) = {prob:.4f} ({prob*100:.2f}%)")

prediction = nb_classifier.predict(test_doc)
print(f"\nPrediction: {prediction.upper()}")

In [ ]:
# Test on all test documents
print("Testing on all test documents:")
print("=" * 70)

test_docs = [text for text, label in test_data]
test_labels = [label for text, label in test_data]

for doc, true_label in test_data:
    pred = nb_classifier.predict(doc)
    probs = nb_classifier.predict_proba(doc)
    status = "✓" if pred == true_label else "✗"
    print(f"{status} '{doc}'")
    print(f"   True: {true_label}, Predicted: {pred}")
    print(f"   Probabilities: pos={probs['positive']:.3f}, neg={probs['negative']:.3f}")
    print()

In [ ]:
# Evaluate the classifier
results = nb_classifier.evaluate(test_docs, test_labels)

print("\nEvaluation Metrics:")
print("=" * 40)
print(f"Accuracy:  {results['accuracy']:.4f} ({results['accuracy']*100:.2f}%)")
print(f"Precision: {results['precision']:.4f}")
print(f"Recall:    {results['recall']:.4f}")
print(f"F1-Score:  {results['f1_score']:.4f}")

---

## Part 4: N-gram Language Models

### 4.1 Theory Review

N-gram models predict the probability of a word based on the previous (n-1) words:

- **Unigram:** $P(w_i)$ - no context
- **Bigram:** $P(w_i|w_{i-1})$ - one word context
- **Trigram:** $P(w_i|w_{i-2}, w_{i-1})$ - two words context

### 4.2 N-gram Extractor

In [ ]:
def extract_ngrams(tokens, n):
    """
    Extract n-grams from a list of tokens.
    
    Parameters:
    -----------
    tokens : list of str
        List of words
    n : int
        N-gram size (1=unigram, 2=bigram, 3=trigram)
    
    Returns:
    --------
    list of tuples
        List of n-grams
    """
    # Add start and end tokens for proper boundary handling
    padded_tokens = ['<s>'] * (n-1) + tokens + ['</s>']
    
    ngrams = []
    for i in range(len(padded_tokens) - n + 1):
        ngram = tuple(padded_tokens[i:i+n])
        ngrams.append(ngram)
    
    return ngrams

# Demonstration
sample_sentence = "I love NLP"
tokens = preprocess_text(sample_sentence)
print(f"Original: '{sample_sentence}'")
print(f"Tokens: {tokens}")
print()

for n in [1, 2, 3]:
    ngrams = extract_ngrams(tokens, n)
    print(f"{n}-grams: {ngrams}")

### 4.3 N-gram Language Model Class

In [ ]:
class NGramLanguageModel:
    """
    N-gram Language Model with Laplace Smoothing
    """
    
    def __init__(self, n=2, k=1):
        """
        Initialize the language model.
        
        Parameters:
        -----------
        n : int
            N-gram order (1=unigram, 2=bigram, 3=trigram)
        k : float
            Smoothing parameter
        """
        self.n = n
        self.k = k
        self.ngram_counts = defaultdict(int)  # count of n-grams
        self.context_counts = defaultdict(int)  # count of (n-1)-grams
        self.vocabulary = set()
        
    def fit(self, documents):
        """
        Train the language model on documents.
        """
        for doc in documents:
            tokens = preprocess_text(doc)
            self.vocabulary.update(tokens)
            
            ngrams = extract_ngrams(tokens, self.n)
            for ngram in ngrams:
                self.ngram_counts[ngram] += 1
                context = ngram[:-1]  # All but last word
                self.context_counts[context] += 1
        
        # Add special tokens to vocabulary
        self.vocabulary.add('<s>')
        self.vocabulary.add('</s>')
        
        print(f"{self.n}-gram model trained!")
        print(f"Vocabulary size: {len(self.vocabulary)}")
        print(f"Unique {self.n}-grams: {len(self.ngram_counts)}")
        
    def probability(self, word, context=None):
        """
        Calculate P(word|context) with smoothing.
        
        For bigram: P(w2|w1) = (count(w1,w2) + k) / (count(w1) + k*|V|)
        """
        if self.n == 1:  # Unigram
            count = self.ngram_counts.get((word,), 0)
            total = sum(self.ngram_counts.values())
            return (count + self.k) / (total + self.k * len(self.vocabulary))
        else:
            if context is None:
                context = tuple(['<s>'] * (self.n - 1))
            ngram = context + (word,)
            ngram_count = self.ngram_counts.get(ngram, 0)
            context_count = self.context_counts.get(context, 0)
            
            return (ngram_count + self.k) / (context_count + self.k * len(self.vocabulary))
    
    def sentence_probability(self, sentence, log_prob=True):
        """
        Calculate the probability of an entire sentence.
        """
        tokens = preprocess_text(sentence)
        ngrams = extract_ngrams(tokens, self.n)
        
        total_log_prob = 0
        probabilities = []
        
        for ngram in ngrams:
            context = ngram[:-1]
            word = ngram[-1]
            prob = self.probability(word, context)
            probabilities.append((ngram, prob))
            total_log_prob += math.log(prob)
        
        if log_prob:
            return total_log_prob, probabilities
        else:
            return math.exp(total_log_prob), probabilities
    
    def perplexity(self, sentence):
        """
        Calculate perplexity of a sentence.
        PPL = exp(-1/N * sum(log P(wi|context)))
        """
        tokens = preprocess_text(sentence)
        ngrams = extract_ngrams(tokens, self.n)
        N = len(ngrams)
        
        log_prob, _ = self.sentence_probability(sentence)
        perplexity = math.exp(-log_prob / N)
        
        return perplexity

print("NGramLanguageModel class defined successfully!")

### 4.4 Training and Comparing N-gram Models

In [ ]:
# Corpus for language model training
corpus = [
    "I love machine learning",
    "I love natural language processing",
    "machine learning is amazing",
    "natural language processing is great",
    "I study data science",
    "data science uses machine learning",
    "deep learning is powerful",
    "I love deep learning",
]

print("Training Corpus:")
for i, sent in enumerate(corpus, 1):
    print(f"  {i}. {sent}")

In [ ]:
# Train different n-gram models
print("Training N-gram Models:")
print("=" * 50)

unigram_model = NGramLanguageModel(n=1, k=1)
unigram_model.fit(corpus)
print()

bigram_model = NGramLanguageModel(n=2, k=1)
bigram_model.fit(corpus)
print()

trigram_model = NGramLanguageModel(n=3, k=1)
trigram_model.fit(corpus)

In [ ]:
# Compare sentence probabilities across models
test_sentences = [
    "I love machine learning",  # Seen in training
    "I love data science",       # Partially seen
    "machine learning is great", # Combination of seen patterns
    "xyz abc def",               # Unseen words
]

print("Sentence Probability Comparison:")
print("=" * 80)

results = []
for sent in test_sentences:
    uni_log, _ = unigram_model.sentence_probability(sent)
    bi_log, _ = bigram_model.sentence_probability(sent)
    tri_log, _ = trigram_model.sentence_probability(sent)
    
    uni_ppl = unigram_model.perplexity(sent)
    bi_ppl = bigram_model.perplexity(sent)
    tri_ppl = trigram_model.perplexity(sent)
    
    results.append({
        'Sentence': sent,
        'Unigram Log P': f"{uni_log:.3f}",
        'Bigram Log P': f"{bi_log:.3f}",
        'Trigram Log P': f"{tri_log:.3f}",
        'Unigram PPL': f"{uni_ppl:.2f}",
        'Bigram PPL': f"{bi_ppl:.2f}",
        'Trigram PPL': f"{tri_ppl:.2f}",
    })

results_df = pd.DataFrame(results)
print(results_df.to_string(index=False))

In [ ]:
# Detailed breakdown for one sentence
example_sentence = "I love machine learning"
print(f"\nDetailed Breakdown for: '{example_sentence}'")
print("=" * 60)

for model, name in [(bigram_model, 'Bigram')]:
    print(f"\n{name} Model:")
    log_prob, probs = model.sentence_probability(example_sentence)
    
    for ngram, prob in probs:
        context = ngram[:-1]
        word = ngram[-1]
        print(f"  P({word}|{context}) = {prob:.4f}")
    
    print(f"  Total log probability: {log_prob:.4f}")
    print(f"  Probability: {math.exp(log_prob):.6f}")
    print(f"  Perplexity: {model.perplexity(example_sentence):.2f}")

---

## Part 5: Combining N-grams with Naive Bayes

### 5.1 N-gram Feature Naive Bayes Classifier

Instead of using just unigrams (single words) as features, we can use bigrams and trigrams to capture word context.

In [ ]:
class NGramNaiveBayes:
    """
    Naive Bayes Classifier using N-gram features.
    Combines unigrams, bigrams, and/or trigrams as features.
    """
    
    def __init__(self, ngram_range=(1, 2), k=1):
        """
        Initialize the classifier.
        
        Parameters:
        -----------
        ngram_range : tuple (min_n, max_n)
            Range of n-grams to use
        k : float
            Smoothing parameter
        """
        self.ngram_range = ngram_range
        self.k = k
        self.class_priors = {}
        self.feature_counts = {}  # {class: {feature: count}}
        self.class_feature_totals = {}
        self.vocabulary = set()  # All unique n-gram features
        self.classes = []
        
    def _extract_features(self, text):
        """
        Extract all n-gram features from text.
        """
        tokens = preprocess_text(text)
        features = []
        
        for n in range(self.ngram_range[0], self.ngram_range[1] + 1):
            ngrams = extract_ngrams(tokens, n)
            # Convert tuples to strings for easier handling
            features.extend(['_'.join(ng) for ng in ngrams])
        
        return features
    
    def fit(self, documents, labels):
        """
        Train the classifier.
        """
        self.classes = list(set(labels))
        n_docs = len(documents)
        
        # Initialize
        for c in self.classes:
            self.feature_counts[c] = defaultdict(int)
            self.class_feature_totals[c] = 0
        
        class_doc_counts = Counter(labels)
        
        # Count features
        for doc, label in zip(documents, labels):
            features = self._extract_features(doc)
            for feature in features:
                self.vocabulary.add(feature)
                self.feature_counts[label][feature] += 1
                self.class_feature_totals[label] += 1
        
        # Calculate priors
        for c in self.classes:
            self.class_priors[c] = class_doc_counts[c] / n_docs
        
        print(f"N-gram Naive Bayes trained!")
        print(f"N-gram range: {self.ngram_range}")
        print(f"Total features (vocabulary): {len(self.vocabulary)}")
        print(f"Class priors: {self.class_priors}")
        
    def _feature_probability(self, feature, class_label):
        """
        Calculate P(feature|class) with smoothing.
        """
        count = self.feature_counts[class_label].get(feature, 0)
        total = self.class_feature_totals[class_label]
        vocab_size = len(self.vocabulary)
        
        return (count + self.k) / (total + self.k * vocab_size)
    
    def predict_proba(self, document):
        """
        Calculate class probabilities.
        """
        features = self._extract_features(document)
        class_scores = {}
        
        for c in self.classes:
            log_prob = math.log(self.class_priors[c])
            for feature in features:
                prob = self._feature_probability(feature, c)
                log_prob += math.log(prob)
            class_scores[c] = log_prob
        
        # Normalize
        max_log = max(class_scores.values())
        exp_scores = {c: math.exp(s - max_log) for c, s in class_scores.items()}
        total = sum(exp_scores.values())
        
        return {c: s / total for c, s in exp_scores.items()}
    
    def predict(self, document):
        """
        Predict class.
        """
        probs = self.predict_proba(document)
        return max(probs, key=probs.get)
    
    def evaluate(self, documents, labels):
        """
        Evaluate classifier.
        """
        predictions = [self.predict(doc) for doc in documents]
        return {
            'accuracy': accuracy_score(labels, predictions),
            'precision': precision_score(labels, predictions, pos_label='positive'),
            'recall': recall_score(labels, predictions, pos_label='positive'),
            'f1_score': f1_score(labels, predictions, pos_label='positive'),
            'predictions': predictions
        }

print("NGramNaiveBayes class defined successfully!")

### 5.2 Comparing Unigram vs Bigram vs Combined Features

In [ ]:
# Extended dataset for better comparison
extended_training = training_data + [
    ("not bad service could be better", "positive"),
    ("not good service could be worse", "negative"),
    ("really great service highly recommend", "positive"),
    ("really bad service do not recommend", "negative"),
    ("food was not great disappointed", "negative"),
    ("food was not bad enjoyed it", "positive"),
]

extended_test = [
    ("great service excellent food", "positive"),
    ("bad experience terrible service", "negative"),
    ("not bad could be better", "positive"),
    ("not good very disappointed", "negative"),
    ("really great highly recommend", "positive"),
    ("really bad do not recommend", "negative"),
]

ext_train_docs = [t for t, l in extended_training]
ext_train_labels = [l for t, l in extended_training]
ext_test_docs = [t for t, l in extended_test]
ext_test_labels = [l for t, l in extended_test]

print(f"Extended training set: {len(extended_training)} documents")
print(f"Extended test set: {len(extended_test)} documents")

In [ ]:
# Train and compare different n-gram configurations
print("Comparing N-gram Configurations:")
print("=" * 60)

configs = [
    ((1, 1), "Unigram only"),
    ((2, 2), "Bigram only"),
    ((3, 3), "Trigram only"),
    ((1, 2), "Unigram + Bigram"),
    ((1, 3), "Unigram + Bigram + Trigram"),
]

comparison_results = []

for ngram_range, name in configs:
    print(f"\nTraining: {name}")
    classifier = NGramNaiveBayes(ngram_range=ngram_range, k=1)
    classifier.fit(ext_train_docs, ext_train_labels)
    
    results = classifier.evaluate(ext_test_docs, ext_test_labels)
    
    comparison_results.append({
        'Configuration': name,
        'N-gram Range': str(ngram_range),
        'Features': len(classifier.vocabulary),
        'Accuracy': f"{results['accuracy']:.3f}",
        'Precision': f"{results['precision']:.3f}",
        'Recall': f"{results['recall']:.3f}",
        'F1-Score': f"{results['f1_score']:.3f}",
    })

print("\n" + "=" * 60)
print("\nComparison Summary:")
comparison_df = pd.DataFrame(comparison_results)
print(comparison_df.to_string(index=False))

In [ ]:
# Visualize comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Metrics comparison
metrics = ['Accuracy', 'Precision', 'Recall', 'F1-Score']
x = np.arange(len(comparison_results))
width = 0.2

for i, metric in enumerate(metrics):
    values = [float(r[metric]) for r in comparison_results]
    axes[0].bar(x + i * width, values, width, label=metric)

axes[0].set_xlabel('Configuration')
axes[0].set_ylabel('Score')
axes[0].set_title('Classification Metrics by N-gram Configuration')
axes[0].set_xticks(x + width * 1.5)
axes[0].set_xticklabels([r['Configuration'] for r in comparison_results], rotation=45, ha='right')
axes[0].legend()
axes[0].set_ylim(0, 1.1)

# Plot 2: Feature count vs F1-Score
features = [r['Features'] for r in comparison_results]
f1_scores = [float(r['F1-Score']) for r in comparison_results]
configs_names = [r['Configuration'] for r in comparison_results]

axes[1].scatter(features, f1_scores, s=100, c='steelblue')
for i, txt in enumerate(configs_names):
    axes[1].annotate(txt, (features[i], f1_scores[i]), 
                     textcoords="offset points", xytext=(5, 5), fontsize=9)
axes[1].set_xlabel('Number of Features')
axes[1].set_ylabel('F1-Score')
axes[1].set_title('Feature Count vs F1-Score')

plt.tight_layout()
plt.savefig('ngram_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

### 5.3 Demonstrating Why Bigrams Help: The "Not" Problem

In [ ]:
# The classic "not good" vs "not bad" problem
negation_examples = [
    "not good service",
    "not bad service",
    "good service",
    "bad service",
]

print("Demonstrating the Negation Problem:")
print("=" * 60)
print("\nUnigram features cannot capture negation context!")

# Show features extracted
print("\nUnigram vs Bigram Features:")
print("-" * 40)

for example in negation_examples:
    tokens = preprocess_text(example)
    unigrams = ['_'.join(ng) for ng in extract_ngrams(tokens, 1)]
    bigrams = ['_'.join(ng) for ng in extract_ngrams(tokens, 2)]
    
    print(f"\n'{example}'")
    print(f"  Unigrams: {unigrams}")
    print(f"  Bigrams:  {bigrams}")

In [ ]:
# Train models and compare predictions
print("\nPrediction Comparison:")
print("=" * 60)

# Train unigram-only model
unigram_nb = NGramNaiveBayes(ngram_range=(1, 1), k=1)
unigram_nb.fit(ext_train_docs, ext_train_labels)

# Train unigram+bigram model
bigram_nb = NGramNaiveBayes(ngram_range=(1, 2), k=1)
bigram_nb.fit(ext_train_docs, ext_train_labels)

print("\nTest sentences with negation:")
test_negation = [
    ("not bad service", "positive"),  # Negation of negative = positive
    ("not good service", "negative"),  # Negation of positive = negative
]

for text, true_label in test_negation:
    uni_pred = unigram_nb.predict(text)
    uni_prob = unigram_nb.predict_proba(text)
    
    bi_pred = bigram_nb.predict(text)
    bi_prob = bigram_nb.predict_proba(text)
    
    print(f"\n'{text}' (True: {true_label})")
    print(f"  Unigram:        {uni_pred:8s} (pos={uni_prob['positive']:.3f}, neg={uni_prob['negative']:.3f})")
    print(f"  Unigram+Bigram: {bi_pred:8s} (pos={bi_prob['positive']:.3f}, neg={bi_prob['negative']:.3f})")

---

## Part 6: Class-Specific N-gram Language Models for Classification

Another approach: Train separate language models for each class and use perplexity for classification.

In [ ]:
class LanguageModelClassifier:
    """
    Classifier using class-specific language models.
    Classifies based on which class's LM assigns lower perplexity.
    """
    
    def __init__(self, n=2, k=1):
        self.n = n
        self.k = k
        self.class_models = {}  # {class: NGramLanguageModel}
        self.classes = []
        
    def fit(self, documents, labels):
        """
        Train a separate language model for each class.
        """
        self.classes = list(set(labels))
        
        # Group documents by class
        class_docs = defaultdict(list)
        for doc, label in zip(documents, labels):
            class_docs[label].append(doc)
        
        # Train LM for each class
        for c in self.classes:
            print(f"\nTraining {self.n}-gram LM for class '{c}':")
            self.class_models[c] = NGramLanguageModel(n=self.n, k=self.k)
            self.class_models[c].fit(class_docs[c])
    
    def predict(self, document):
        """
        Predict using perplexity: lower perplexity = more likely class.
        """
        perplexities = {}
        for c in self.classes:
            perplexities[c] = self.class_models[c].perplexity(document)
        
        # Return class with lowest perplexity
        return min(perplexities, key=perplexities.get)
    
    def predict_with_scores(self, document):
        """
        Return prediction with perplexity scores.
        """
        perplexities = {}
        for c in self.classes:
            perplexities[c] = self.class_models[c].perplexity(document)
        
        prediction = min(perplexities, key=perplexities.get)
        return prediction, perplexities

print("LanguageModelClassifier class defined successfully!")

In [ ]:
# Train the LM-based classifier
print("Training Language Model Classifier:")
print("=" * 60)

lm_classifier = LanguageModelClassifier(n=2, k=1)
lm_classifier.fit(ext_train_docs, ext_train_labels)

In [ ]:
# Test the LM classifier
print("\nLanguage Model Classifier Predictions:")
print("=" * 70)

for text, true_label in extended_test:
    pred, perplexities = lm_classifier.predict_with_scores(text)
    status = "✓" if pred == true_label else "✗"
    
    print(f"\n{status} '{text}'")
    print(f"   True: {true_label}, Predicted: {pred}")
    print(f"   Perplexity - positive: {perplexities['positive']:.2f}, negative: {perplexities['negative']:.2f}")
    print(f"   (Lower perplexity = more similar to class language)")

---

## Part 7: Using Sklearn for Comparison

In [ ]:
# Compare with sklearn's implementation
print("Sklearn Naive Bayes Comparison:")
print("=" * 60)

sklearn_results = []

for ngram_range, name in [((1, 1), "Unigram"), ((1, 2), "Unigram+Bigram"), ((1, 3), "Uni+Bi+Tri")]:
    # Create vectorizer
    vectorizer = CountVectorizer(ngram_range=ngram_range)
    
    # Transform data
    X_train = vectorizer.fit_transform(ext_train_docs)
    X_test = vectorizer.transform(ext_test_docs)
    
    # Train classifier
    clf = MultinomialNB(alpha=1.0)  # alpha=1 is Laplace smoothing
    clf.fit(X_train, ext_train_labels)
    
    # Evaluate
    y_pred = clf.predict(X_test)
    
    sklearn_results.append({
        'Configuration': name,
        'Features': len(vectorizer.vocabulary_),
        'Accuracy': f"{accuracy_score(ext_test_labels, y_pred):.3f}",
        'F1-Score': f"{f1_score(ext_test_labels, y_pred, pos_label='positive'):.3f}",
    })
    
    print(f"\n{name}:")
    print(f"  Features: {len(vectorizer.vocabulary_)}")
    print(f"  Sample features: {list(vectorizer.vocabulary_.keys())[:10]}...")

print("\n" + "=" * 60)
print("\nSklearn Results Summary:")
print(pd.DataFrame(sklearn_results).to_string(index=False))

---

## Part 8: Exercises

### Exercise 1: Implement Add-k Smoothing Comparison

Compare classifier performance with different smoothing values (k=0.1, 0.5, 1.0, 2.0).

In [ ]:
# Exercise 1: Your code here
# Hint: Create classifiers with different k values and compare their F1 scores

smoothing_values = [0.1, 0.5, 1.0, 2.0]

# TODO: Implement comparison
# for k in smoothing_values:
#     classifier = NGramNaiveBayes(ngram_range=(1, 2), k=k)
#     ...

print("Exercise 1: Implement smoothing comparison")

### Exercise 2: Create a Nepali Sentiment Dataset

Create a small sentiment analysis dataset using Nepali business reviews and test the classifier.

In [ ]:
# Exercise 2: Your code here
# Create Nepali sentiment data (you can use romanized Nepali)

nepali_data = [
    # Add your Nepali examples here
    # ("ramro sewa", "positive"),
    # ("naramro khana", "negative"),
]

print("Exercise 2: Create Nepali sentiment dataset")

### Exercise 3: Perplexity Analysis

Calculate and compare the perplexity of different sentences using your trained language models.

In [ ]:
# Exercise 3: Your code here
# Test sentences:
test_sentences_ex3 = [
    "I love learning new things",
    "machine learning is fun",
    "random words here there",
    "the quick brown fox",
]

# TODO: Calculate perplexity for each sentence using bigram_model
# Which sentences have lower perplexity and why?

print("Exercise 3: Analyze perplexity")

### Exercise 4: Feature Importance Analysis

Find the most discriminative n-gram features for each class.

In [ ]:
# Exercise 4: Your code here
# Hint: Compare P(feature|positive) vs P(feature|negative)
# Features with largest difference are most discriminative

def find_discriminative_features(classifier, top_n=10):
    """
    Find the most discriminative features for each class.
    """
    # TODO: Implement this function
    pass

print("Exercise 4: Find discriminative features")

---

## Summary

### Key Takeaways:

1. **Naive Bayes** is a simple but effective probabilistic classifier based on Bayes' theorem

2. **Laplace Smoothing** prevents zero probabilities for unseen words

3. **N-gram Language Models** capture word context:
   - Unigrams: No context (bag of words)
   - Bigrams: One word context
   - Trigrams: Two words context

4. **Combining N-grams with Naive Bayes** improves classification by:
   - Capturing word order and phrases
   - Handling negation better ("not good" vs "good")
   - Providing richer feature representation

5. **Trade-offs**:
   - Higher n-grams = more context but more sparsity
   - More features = potentially better performance but risk of overfitting

### Further Reading:
- Jurafsky & Martin, "Speech and Language Processing" - Chapters on Naive Bayes and N-grams
- Manning et al., "Introduction to Information Retrieval" - Text Classification

In [ ]:
print("Lab completed! Great work! 🎉")